<a href="https://colab.research.google.com/github/LKai22/007022/blob/master/Reach_static.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#라이브러리 설치

In [ ]:
!pip install pymysql

In [ ]:
import numpy as np
import pandas as pd
import math
import pymysql
from datetime import datetime
import json
# 최대 줄 수 설정
pd.set_option('display.max_rows', 10000)
# 최대 열 수 설정
pd.set_option('display.max_columns', 10000)
# 표시할 가로의 길이
pd.set_option('display.width', 10000)

#1.라걸 1차 6월 18일 ~ 8월 5일

In [7]:
conn = pymysql.connect(host='prod-dalgo-db.cluster-ro-ce4m6idyzrne.ap-northeast-2.rds.amazonaws.com',
                       user='dalgo_readonly',
                       password='81HiEYRl9MlfRk8Vi9rM',
                       db='dalgo_data',
                       charset='utf8')

In [8]:
cursor = conn.cursor()
sql = "SELECT * FROM dalgo_data.daily_user_pattern where createdAt BETWEEN '2020-06-18' AND '2020-08-06';"
cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=['patternId', 'createdAt', 'updatedAt', 'dataType','drivingDay','dataJson','userId'])
doiscode = pd.read_csv('/content/drive/My Drive/Colab/Data Preprocessing/data/df_emd_new.csv')
doiscode['EMD_CD'] = doiscode['EMD_CD'].astype(str)

In [9]:
campaign_72 = [2,32,47,60,72,86,125,192,263,268,
               269,271,272,273,279,281,296,299,
               300,309,310,314,315,316,323,327,
               333,336,338,339,343,349,352,357,
               364,369,382,386,396,427,440,442,
               444,446,447,456,457,486,492,498,
               502,520,576,606,614,634,635,637,
               639,640,643,653,655,657,659,661,
               663,664,666,668,671,673,674,676,
               681,683,684,692,697,698,700,704,
               717,722,724,725,728,729,731,732,
               734,737,738,751,754,764,765,766,
               769,770,774,775,782,785,789,794,
               796,800,801,804,807,810,814,816,
               820,827,829,830,832,838,839,841,
               846,848,851,857,862,863,864,865,
               867,869,870,871,875,877,881,882,
               886,897,899,902,904,910,912,913,
               923,925,926,929,930,931,932,933,
               935,936,939,943,944,955,957,960,
               962,966,968,978,979,986,987,990,
               992,996,997,998,1008,1013,1018,1021,
               1027,1028,1035,1049,1053,1059,1064,
               1072,1077,1080,1084,1087,1095,1101,
               1103,1108,1109,1121,1124,1135,1142,
               1146,1149,1153,1156,1159,1160,1161,
               1163,1166,1168,1169,1171,1174,1175,
               1177,1178,1186,1190,1191,1194,1203,
               1205,1210,1217,1218,1222,1223,1231,
               1233,1236,1237,1241,1242,1247,1252,
               1254,1266,1268,1282,1285,1286,1292,
               1293,1302,1304,1307,1311,1313,1315,
               1321,1323,1326,1327,1329,1332,1344,
               1346,1351,1358,1365,1367]
               
campaign_72 = sorted(campaign_72)

In [10]:
lat_long_list=[]
reach_list=[]
user_Id = []

for uID in campaign_72:
    data_1 = df.loc[df['userId'] == uID]
    data_2 = data_1['dataJson'].loc[df['dataType'] == 'Reach']
    for key, value in data_2.items():
        dict_values=json.loads(value)
        for key, value in dict_values.items():
            if key != '00000000' and value != 0:
                user_Id.append(uID)
                lat_long_list.append(key)
                reach_list.append(value)
                # print(key, ":", value)

df_r=pd.DataFrame(user_Id, columns=['주행자'])
df_r['행정동코드'] = lat_long_list
df_r['도달'] = reach_list

sd_list = []
skk_list = []
emd_list = []

for i in df_r['행정동코드']:
  for idx, row in doiscode['EMD_CD'].iteritems():
    if i == row[0:8]:
      sd_list.append(doiscode.loc[idx, 'SD_KOR_NM'])
      skk_list.append(doiscode.loc[idx, 'SKK_KOR_NM'])
      emd_list.append(doiscode.loc[idx, 'EMD_KOR_NM'])
      
df_r['시도']=sd_list
df_r['시군구']=skk_list
df_r['읍면동']=emd_list

##1.1 Reach
### 시도, 시군구 별 각각의 도달의 통계량


*   count -> 몇번 주행하였는지 (시군구별 주행자의 주행 합산 회수)
*   max -> 하루 도달의 최고치
*   min -> 하루 도달의 최소치
*   sum -> 시군구별 합
*   mena -> 각가의 시군구별 1일 평균 도달수






In [52]:
g_r1=round(df_r.groupby(['시도','시군구'])['도달'].agg(['count','max','min','sum','std']),0)
g_r1['mean']=round(g_r1['sum'] / 48,0) #48일로 나눔
g_r1.sort_values('mean', ascending=False) # 각각의 시군구별 1일 평균 도달수
# 서울, 부산 상위 5개만 쓰시면 될듯

count       max      min        sum      std     mean
시도      시군구                                                             
서울특별시   서초구         1467   49083.0      0.0  2500221.0   3957.0  52088.0
        관악구          754   88231.0      3.0  1781733.0   5608.0  37119.0
        강남구         1177   80226.0      0.0  1690277.0   4587.0  35214.0
        영등포구        1823   43316.0      1.0  1564011.0   2823.0  32584.0
        송파구          887   47040.0      5.0  1520279.0   4660.0  31672.0
경기도     화성시         1228   87190.0      0.0  1467919.0   3401.0  30582.0
서울특별시   구로구          901   26311.0      0.0  1420818.0   3199.0  29600.0
        은평구          667   89146.0      7.0  1365653.0   6452.0  28451.0
경기도     용인시 기흥구      997   90133.0      0.0  1295879.0   4444.0  26997.0
        부천시          841   36049.0      0.0  1272585.0   3243.0  26512.0
        광명시          603   89612.0      2.0  1216287.0   6435.0  25339.0
        광주시          723   37178.0      0.0  1177662.0   3754.0  24535.0
서울특별시   노원구          663   19797.0      0.0   989108.0   2609.0  20606.0
경기도     고양시 덕양구      730   72106.0      0.0   972368.0   5160.0  20258.0
서울특별시   중랑구          596   32912.0      1.0   960338.0   3621.0  20007.0
경기도     평택시          467   52886.0      2.0   939697.0   4298.0  19577.0
서울특별시   강서구          643   74151.0      1.0   928213.0   5384.0  19338.0
경기도     의정부시         525   63701.0      0.0   917552.0   5023.0  19116.0
부산광역시   부산진구         679   22240.0      6.0   893701.0   2448.0  18619.0
경기도     성남시 분당구      669   48316.0      1.0   892015.0   3639.0  18584.0
서울특별시   성동구          898   46277.0      0.0   888364.0   3622.0  18508.0
        동작구          877   33436.0      1.0   881221.0   2458.0  18359.0
        양천구          779   23336.0      3.0   859399.0   2636.0  17904.0
        동대문구         823   26567.0      0.0   826083.0   2556.0  17210.0
인천광역시   미추홀구         280   69687.0      6.0   758865.0   5937.0  15810.0
서울특별시   용산구          852   58110.0      0.0   741165.0   3388.0  15441.0
경기도     용인시 수지구      421   35385.0      0.0   722031.0   3991.0  15042.0
        안양시 만안구      330   49329.0      0.0   685709.0   4276.0  14286.0
서울특별시   성북구          767   77013.0      0.0   676614.0   3975.0  14096.0
경기도     남양주시         466   44085.0      0.0   666855.0   3795.0  13893.0
        수원시 장안구      345   25857.0      2.0   661587.0   3117.0  13783.0
        파주시          405   33753.0      0.0   660001.0   3476.0  13750.0
서울특별시   마포구          904   58800.0      0.0   659115.0   3395.0  13732.0
        서대문구         696   22941.0      1.0   650266.0   2638.0  13547.0
        중구           751   86768.0      4.0   649220.0   3936.0  13525.0
        광진구          696   28339.0      1.0   642568.0   2604.0  13387.0
인천광역시   서구           459   22229.0      6.0   626024.0   2959.0  13042.0
서울특별시   강동구          354   28680.0      3.0   618851.0   3446.0  12893.0
경기도     수원시 영통구      576   20806.0      2.0   607289.0   2252.0  12652.0
        고양시 일산동구     438   45498.0      1.0   542547.0   4003.0  11303.0
서울특별시   강북구          243   15474.0     26.0   522361.0   3034.0  10883.0
부산광역시   수영구          236   31868.0      0.0   521391.0   3712.0  10862.0
서울특별시   종로구          876   72487.0      3.0   509888.0   3036.0  10623.0
경기도     수원시 팔달구      343   42170.0      1.0   485188.0   3988.0  10108.0
부산광역시   북구           192   19187.0      1.0   462434.0   3387.0   9634.0
        남구           238  131570.0      1.0   443879.0   9670.0   9247.0
경기도     수원시 권선구      438   24479.0      1.0   428992.0   2678.0   8937.0
인천광역시   중구           171  127011.0     22.0   427997.0  10836.0   8917.0
경기도     성남시 수정구      389   30181.0      1.0   383696.0   2696.0   7994.0
부산광역시   사하구          296   13392.0      3.0   383527.0   2333.0   7990.0
경기도     구리시          387   42070.0      0.0   378544.0   3527.0   7886.0
        고양시 일산서구     274   54581.0      0.0   375623.0   3948.0   7825.0
        안산시 상록구      210   16957.0      0.0   364579.0   3293.0   7595.0
서울특별

In [35]:
g_r2=round(df_r.groupby('시도')['도달'].agg(['sum']),0)
g_r2.sort_values('sum', ascending=False)

,sum
시도,
서울특별시,24399857.0
경기도,20724797.0
부산광역시,5045460.0
인천광역시,2809878.0
경상남도,1098330.0
강원도,891928.0
충청남도,593059.0
대구광역시,568019.0
경상북도,389727.0


##1.2 Exposure
### 시도, 시군구 별 각각의 도달의 통계량


*   count -> 몇번 주행하였는지 (시군구별 주행자의 주행 합산 회수)
*   max -> 하루 노출의 최고치
*   min -> 하루 노출의 최소치
*   sum -> 시군구별 합
*   mena -> 각가의 시군구별 1일 평균 도달수






In [36]:
exposure_lat_long_list=[]
exposure_list=[]
exposure_user_Id = []


for uID in campaign_72:
    data_1 = df.loc[df['userId'] == uID]
    data_2 = data_1['dataJson'].loc[df['dataType'] == 'Exposure']
    for key, value in data_2.items():
        dict_values=json.loads(value)
        for key, value in dict_values.items():
            if key != '00000000' and value != 0:
                exposure_user_Id.append(uID)
                exposure_lat_long_list.append(key)
                exposure_list.append(value)
                # print(key, ":", value)

df_e=pd.DataFrame(exposure_user_Id, columns=['주행자'])
df_e['행정동코드'] = exposure_lat_long_list
df_e['노출'] = exposure_list

ex_sd_list = []
ex_skk_list = []
ex_emd_list = []

for i in df_e['행정동코드']:
  for idx, row in doiscode['EMD_CD'].iteritems():
    if i == row[0:8]:
      ex_sd_list.append(doiscode.loc[idx, 'SD_KOR_NM'])
      ex_skk_list.append(doiscode.loc[idx, 'SKK_KOR_NM'])
      ex_emd_list.append(doiscode.loc[idx, 'EMD_KOR_NM'])

df_e['시도']=ex_sd_list
df_e['시군구']=ex_skk_list
df_e['읍면동']=ex_emd_list

In [44]:
g_e1=round(df_e.groupby(['시도','시군구'])['노출'].agg(['count','max','min','sum','std']),0)
g_e1['mean']=round(g_e1['sum'] / 48,0) #48일로 나눔
g_e1.sort_values('mean', ascending=False) # 각각의 시군구별 1일 평균 도달수
# 서울, 부산 상위 5개만 쓰시면 될듯

count       max      min        sum      std      mean
시도      시군구                                                              
서울특별시   서초구         1468   49083.0      0.0  5382092.0   7077.0  112127.0
        관악구          755   88363.0      7.0  3840765.0   8514.0   80016.0
경기도     화성시         1257   87190.0      0.0  3483833.0   5728.0   72580.0
서울특별시   구로구          902   32752.0      0.0  3275512.0   6378.0   68240.0
        강남구         1178   80230.0      0.0  3160403.0   6024.0   65842.0
경기도     부천시          841   41531.0      0.0  2983899.0   6369.0   62165.0
서울특별시   영등포구        1824   45673.0      1.0  2981520.0   4130.0   62115.0
        송파구          887   47040.0      5.0  2970108.0   6608.0   61877.0
        노원구          663   29268.0      0.0  2683091.0   6350.0   55898.0
경기도     용인시 기흥구     1002   91792.0      0.0  2670543.0   6810.0   55636.0
서울특별시   은평구          669   89241.0      7.0  2637466.0   8410.0   54947.0
경기도     광주시          739   37981.0      1.0  2571281.0   6885.0   53568.0
        광명시          603   89612.0      2.0  2483938.0   8401.0   51749.0
부산광역시   부산진구         679   32692.0      6.0  2201950.0   5478.0   45874.0
서울특별시   양천구          780   35264.0      3.0  2090728.0   5279.0   43557.0
경기도     평택시          468   54474.0      2.0  2069747.0   7015.0   43120.0
서울특별시   동대문구         824   40201.0      0.0  2015595.0   5966.0   41992.0
        중랑구          599   37488.0      1.0  1978721.0   6168.0   41223.0
경기도     의정부시         529   63701.0      0.0  1834571.0   6677.0   38220.0
        성남시 분당구      671   59025.0      1.0  1800624.0   5785.0   37513.0
서울특별시   동작구          877   33436.0      1.0  1740468.0   4359.0   36260.0
경기도     수원시 영통구      576   28460.0      2.0  1725183.0   5938.0   35941.0
서울특별시   용산구          853   68963.0      0.0  1645821.0   5389.0   34288.0
경기도     안양시 만안구      330   54791.0      0.0  1630485.0   7333.0   33968.0
        파주시          420   34255.0      1.0  1626486.0   7071.0   33885.0
인천광역시   미추홀구         280   72971.0      6.0  1622873.0   8629.0   33810.0
서울특별시   성북구          768  256572.0      0.0  1606824.0  10429.0   33476.0
경기도     남양주시         491   51962.0      0.0  1603829.0   6675.0   33413.0
인천광역시   서구           459   40138.0      6.0  1581844.0   6450.0   32955.0
서울특별시   성동구          898   50346.0      0.0  1567178.0   4906.0   32650.0
경기도     수원시 장안구      345   34837.0      2.0  1559842.0   7526.0   32497.0
        고양시 덕양구      743   72395.0      0.0  1529482.0   6409.0   31864.0
서울특별시   강북구          243   28643.0     26.0  1524446.0   8600.0   31759.0
        강서구          643   76266.0      1.0  1509940.0   6291.0   31457.0
경기도     용인시 수지구      422   35473.0      0.0  1476683.0   6266.0   30764.0
서울특별시   광진구          696   34470.0      1.0  1433014.0   4651.0   29854.0
        강동구          354   29284.0      3.0  1322573.0   6354.0   27554.0
        서대문구         696   31884.0      3.0  1321429.0   4732.0   27530.0
부산광역시   북구           193   32947.0      1.0  1169735.0   7919.0   24369.0
        수영구          236   31868.0      0.0  1148292.0   7169.0   23923.0
서울특별시   마포구          905   58800.0      0.0  1096981.0   4175.0   22854.0
경기도     수원시 권선구      439   34804.0      1.0  1055814.0   5453.0   21996.0
부산광역시   기장군          291   27375.0      1.0  1032141.0   5738.0   21503.0
경기도     고양시 일산서구     278   55054.0      0.0  1004673.0   6482.0   20931.0
        고양시 일산동구     445   45833.0      3.0   999677.0   5233.0   20827.0
서울특별시   중구           751   86853.0      4.0   996724.0   4532.0   20765.0
        종로구          876   76799.0      3.0   986877.0   3904.0   20560.0
경기도     성남시 수정구      389   36867.0      1.0   980173.0   5409.0   20420.0
        김포시          289   28965.0      1.0   908571.0   6340.0   18929.0
부산광역시   사하구          297   20266.0      3.0   891123.0   5094.0   18565.0
경기도     시흥시          519   32856.0      0.0   886073.0   4545.0   18460.0
        구리시          393   42070.0      0.0   837586.0   5583.0   17450.0
        수원시 팔달구      34

In [47]:
g_e2=round(df_e.groupby('시도')['노출'].agg(['sum']),0)
g_e2.sort_values('sum', ascending=False)

,sum
시도,
서울특별시,50946197.0
경기도,45641518.0
부산광역시,11425827.0
인천광역시,5743301.0
강원도,2153362.0
경상남도,2038346.0
대구광역시,1332744.0
충청남도,1185492.0
경상북도,951994.0


#2.라걸 2차 7월 27일 ~ 8월 22일

In [48]:
cursor = conn.cursor()
sql = "SELECT * FROM dalgo_data.daily_user_pattern where createdAt BETWEEN '2020-07-27' AND '2020-08-24';"
cursor.execute(sql)
result = cursor.fetchall()
df2 = pd.DataFrame(result, columns=['patternId', 'createdAt', 'updatedAt', 'dataType','drivingDay','dataJson','userId'])
doiscode2 = pd.read_csv('/content/drive/My Drive/Colab/Data Preprocessing/data/df_emd_new.csv')
doiscode2['EMD_CD'] = doiscode2['EMD_CD'].astype(str)

In [54]:
campaign_101 = [2197, 635, 1159, 1149, 2115,
                296, 234, 2388, 2400,2423,2433,
                2438,2501,2519,2526,2464,2560,
                2616,2617,2471,2654,2660,2666,
                2545,2681,2427,2619,2553,2707,
                2705,2521,785,2516,1327,2454,1304,
                2734,276,1236,1190,2739,2740,2741,
                2747,1483,992,1498,1459,585,1699,
                2753,343,882,1166
]
campaign_101 = sorted(campaign_101)

In [55]:
lat_long_list=[]
reach_list=[]
user_Id = []

for uID in campaign_101:
    data_1 = df2.loc[df2['userId'] == uID]
    data_2 = data_1['dataJson'].loc[df2['dataType'] == 'Reach']
    for key, value in data_2.items():
        dict_values=json.loads(value)
        for key, value in dict_values.items():
            if key != '00000000' and value != 0:
                user_Id.append(uID)
                lat_long_list.append(key)
                reach_list.append(value)
                # print(key, ":", value)

df_r=pd.DataFrame(user_Id, columns=['주행자'])
df_r['행정동코드'] = lat_long_list
df_r['도달'] = reach_list

sd_list = []
skk_list = []
emd_list = []

for i in df_r['행정동코드']:
  for idx, row in doiscode['EMD_CD'].iteritems():
    if i == row[0:8]:
      sd_list.append(doiscode.loc[idx, 'SD_KOR_NM'])
      skk_list.append(doiscode.loc[idx, 'SKK_KOR_NM'])
      emd_list.append(doiscode.loc[idx, 'EMD_KOR_NM'])
      
df_r['시도']=sd_list
df_r['시군구']=skk_list
df_r['읍면동']=emd_list

##2.1 Reach
### 시도, 시군구 별 각각의 도달의 통계량


*   count -> 몇번 주행하였는지 (시군구별 주행자의 주행 합산 회수)
*   max -> 하루 도달의 최고치
*   min -> 하루 도달의 최소치
*   sum -> 시군구별 합
*   mena -> 각가의 시군구별 1일 평균 도달수






In [57]:
g_r1=round(df_r.groupby(['시도','시군구'])['도달'].agg(['count','max','min','sum','std']),0)
g_r1['mean']=round(g_r1['sum'] / 28,0) #48일로 나눔
g_r1.sort_values('mean', ascending=False) # 각각의 시군구별 1일 평균 도달수
# 서울, 부산 상위 5개만 쓰시면 될듯

count      max     min       sum     std     mean
시도      시군구                                                        
경기도     파주시         246  62104.0     0.0  324303.0  4312.0  11582.0
        안산시 단원구     106  30093.0    17.0  303475.0  3878.0  10838.0
        화성시         135  84010.0    14.0  292966.0  7492.0  10463.0
인천광역시   부평구          90  76185.0    20.0  290976.0  8940.0  10392.0
경기도     김포시         123  20471.0     3.0  282594.0  3742.0  10093.0
        안성시         141  13579.0     0.0  278808.0  2712.0   9957.0
충청남도    아산시         111  17293.0     0.0  251180.0  3959.0   8971.0
경기도     군포시          65  18294.0     2.0  221897.0  4221.0   7925.0
서울특별시   양천구          58  59505.0     4.0  215217.0  8959.0   7686.0
        노원구          53  23466.0    16.0  194273.0  5572.0   6938.0
경기도     용인시 수지구      46  26993.0    18.0  161955.0  5711.0   5784.0
인천광역시   남동구          89  22693.0     5.0  161925.0  3996.0   5783.0
경기도     오산시         108  11982.0     0.0  160835.0  2958.0   5744.0
부산광역시   영도구          63  39053.0    31.0  159407.0  5532.0   5693.0
경기도     남양주시         59  38917.0     1.0  151770.0  6461.0   5420.0
인천광역시   서구           98  31371.0     1.0  150746.0  4363.0   5384.0
전라북도    전주시 완산구      89   9558.0     0.0  145997.0  2575.0   5214.0
서울특별시   중랑구          62  21350.0    11.0  135862.0  4011.0   4852.0
경기도     수원시 영통구      53   8494.0    17.0  134519.0  2441.0   4804.0
서울특별시   강북구          29   8772.0    31.0  129196.0  2821.0   4614.0
강원도     횡성군          43  12142.0     8.0  123967.0  2827.0   4427.0
경기도     고양시 덕양구     124  23801.0     6.0  117878.0  2710.0   4210.0
        안산시 상록구      44  23336.0     1.0  114745.0  4057.0   4098.0
서울특별시   서초구         122  19984.0     7.0  112860.0  2678.0   4031.0
경기도     광주시          46  10785.0     5.0  110494.0  2923.0   3946.0
서울특별시   영등포구         97  28437.0    12.0  103296.0  3634.0   3689.0
인천광역시   계양구          78   8106.0    10.0   94441.0  2207.0   3373.0
광주광역시   광산구         105  19388.0     2.0   93422.0  2782.0   3336.0
서울특별시   금천구          26  13660.0    13.0   85965.0  3455.0   3070.0
부산광역시   사상구          24   7256.0    34.0   77684.0  1844.0   2774.0
경기도     구리시          37  42093.0    11.0   75831.0  7202.0   2708.0
충청북도    진천군          26  16698.0     7.0   71064.0  4445.0   2538.0
경기도     여주시          32   5274.0    11.0   66949.0  2029.0   2391.0
서울특별시   송파구          60   7733.0    14.0   64721.0  1824.0   2311.0
충청남도    천안시 서북구      33   6889.0    17.0   63100.0  2166.0   2254.0
경기도     평택시          46  12342.0     6.0   60380.0  2463.0   2156.0
        의정부시         58   3676.0     5.0   57208.0  1280.0   2043.0
전라남도    목포시          36  16212.0     3.0   50073.0  2984.0   1788.0
경기도     용인시 기흥구      41  36024.0     2.0   49439.0  5670.0   1766.0
서울특별시   동대문구         16  21696.0    32.0   45832.0  5802.0   1637.0
경기도     부천시          87   6111.0     6.0   37688.0  1193.0   1346.0
서울특별시   동작구          47   3872.0    27.0   36869.0  1323.0   1317.0
경기도     수원시 권선구      61  10142.0     2.0   36744.0  1934.0   1312.0
인천광역시   연수구          30   5074.0     4.0   34917.0  1514.0   1247.0
충청북도    청주시 흥덕구      21   6377.0     7.0   34692.0  1908.0   1239.0
경기도     연천군           5  22295.0    33.0   30960.0  9507.0   1106.0
서울특별시   강남구          72   3898.0     2.0   26591.0   758.0    950.0
경기도     고양시 일산동구    113   9613.0     3.0   24289.0   990.0    867.0
경상남도    거제시          27   7680.0    17.0   24145.0  1856.0    862.0
경기도     과천시          31   4415.0     5.0   23224.0  1387.0    829.0
서울특별시   구로구          54   3819.0     3.0   22175.0   852.0    792.0
        강서구          39   4479.0    21.0   18783.0  1175.0    671.0
        중구           68   4452.0    12.0   18639.0   622.0    666.0
강원도     홍천군           7   8824.0    11.0   16303.0  3422.0    582.0
서울특별시   성북구          83   2447.0     3.0   13358.0   357.0    477.0
전라북도    전주시 덕진구      97   3224.0     7.0   12404.0   354.0    443.0
서울특별시   종로구          80   2240.0    22.0   10937.0   359.0    391.0
경기도   

In [60]:
g_r2=round(df_r.groupby('시도')['도달'].agg(['sum']),0)
g_r2.sort_values('sum', ascending=False)

,sum
시도,
경기도,3184790.0
서울특별시,1259117.0
인천광역시,736607.0
충청남도,316772.0
부산광역시,241230.0
전라북도,169275.0
강원도,154312.0
충청북도,116669.0
광주광역시,96461.0


##2.2 Exposure
### 시도, 시군구 별 각각의 도달의 통계량


*   count -> 몇번 주행하였는지 (시군구별 주행자의 주행 합산 회수)
*   max -> 하루 노출의 최고치
*   min -> 하루 노출의 최소치
*   sum -> 시군구별 합
*   mena -> 각가의 시군구별 1일 평균 도달수






In [62]:
exposure_lat_long_list=[]
exposure_list=[]
exposure_user_Id = []


for uID in campaign_101:
    data_1 = df2.loc[df2['userId'] == uID]
    data_2 = data_1['dataJson'].loc[df2['dataType'] == 'Exposure']
    for key, value in data_2.items():
        dict_values=json.loads(value)
        for key, value in dict_values.items():
            if key != '00000000' and value != 0:
                exposure_user_Id.append(uID)
                exposure_lat_long_list.append(key)
                exposure_list.append(value)
                # print(key, ":", value)

df_e=pd.DataFrame(exposure_user_Id, columns=['주행자'])
df_e['행정동코드'] = exposure_lat_long_list
df_e['노출'] = exposure_list

ex_sd_list = []
ex_skk_list = []
ex_emd_list = []

for i in df_e['행정동코드']:
  for idx, row in doiscode['EMD_CD'].iteritems():
    if i == row[0:8]:
      ex_sd_list.append(doiscode.loc[idx, 'SD_KOR_NM'])
      ex_skk_list.append(doiscode.loc[idx, 'SKK_KOR_NM'])
      ex_emd_list.append(doiscode.loc[idx, 'EMD_KOR_NM'])

df_e['시도']=ex_sd_list
df_e['시군구']=ex_skk_list
df_e['읍면동']=ex_emd_list

In [63]:
g_e1=round(df_e.groupby(['시도','시군구'])['노출'].agg(['count','max','min','sum','std']),0)
g_e1['mean']=round(g_e1['sum'] / 48,0) #48일로 나눔
g_e1.sort_values('mean', ascending=False) # 각각의 시군구별 1일 평균 도달수
# 서울, 부산 상위 5개만 쓰시면 될듯

count      max      min       sum      std     mean
시도      시군구                                                          
경기도     파주시         251  65451.0      3.0  940541.0   7171.0  19595.0
        김포시         125  33195.0      3.0  798137.0   9402.0  16628.0
        군포시          65  32446.0      2.0  661051.0  11811.0  13772.0
        안성시         142  24890.0      0.0  639666.0   6152.0  13326.0
        화성시         135  84010.0     14.0  612206.0   8992.0  12754.0
        안산시 단원구     106  30093.0     17.0  581191.0   6526.0  12108.0
인천광역시   부평구          90  77887.0     20.0  565515.0  11176.0  11782.0
충청남도    아산시         112  30999.0      0.0  476831.0   7149.0   9934.0
서울특별시   양천구          58  59505.0      4.0  454180.0  12480.0   9462.0
        강북구          29  29337.0     31.0  453745.0   9613.0   9453.0
부산광역시   영도구          63  49779.0     31.0  414721.0  10051.0   8640.0
강원도     횡성군          44  29438.0     17.0  401437.0   8318.0   8363.0
서울특별시   노원구          53  28808.0     27.0  384413.0   8716.0   8009.0
인천광역시   서구           98  40715.0      1.0  383912.0   7634.0   7998.0
서울특별시   중랑구          62  29966.0     11.0  348342.0   8333.0   7257.0
경기도     광주시          57  28474.0      6.0  346181.0   8449.0   7212.0
        용인시 수지구      46  31844.0     18.0  317703.0   8595.0   6619.0
전라북도    전주시 완산구      89  21780.0      0.0  316991.0   5506.0   6604.0
경기도     고양시 덕양구     128  32317.0      6.0  301812.0   5624.0   6288.0
인천광역시   남동구          89  28832.0      5.0  288218.0   6554.0   6005.0
        계양구          78  27002.0     10.0  276925.0   6813.0   5769.0
경기도     오산시         109  26924.0     21.0  270702.0   5318.0   5640.0
서울특별시   서초구         122  20267.0      7.0  269320.0   4880.0   5611.0
경기도     수원시 영통구      53  24775.0     17.0  262886.0   5564.0   5477.0
        안산시 상록구      44  25433.0      1.0  259323.0   7258.0   5403.0
서울특별시   금천구          26  28545.0     13.0  258796.0   9056.0   5392.0
부산광역시   사상구          24  24368.0     34.0  250903.0   6569.0   5227.0
광주광역시   광산구         105  23266.0      2.0  221026.0   4565.0   4605.0
경기도     남양주시         59  39595.0      1.0  217651.0   7195.0   4534.0
서울특별시   영등포구         97  33350.0     12.0  214163.0   4996.0   4462.0
경기도     의정부시         58  12397.0      5.0  190711.0   4324.0   3973.0
서울특별시   송파구          60  13234.0     14.0  172646.0   4561.0   3597.0
경기도     평택시          48  26817.0      6.0  156014.0   5815.0   3250.0
        여주시          32  15830.0     20.0  142770.0   4876.0   2974.0
        부천시          87  20373.0      6.0  121891.0   3992.0   2539.0
충청북도    진천군          30  16698.0      8.0  121175.0   6078.0   2524.0
서울특별시   동작구          47  12881.0     27.0  120037.0   4426.0   2501.0
전라남도    목포시          38  16212.0      3.0  116247.0   4366.0   2422.0
서울특별시   동대문구         16  40201.0     32.0  116185.0  13124.0   2421.0
경기도     구리시          37  42093.0     11.0  115140.0   8064.0   2399.0
충청북도    청주시 흥덕구      21  21422.0      7.0  107045.0   6102.0   2230.0
충청남도    천안시 서북구      34  14802.0     17.0  106154.0   3557.0   2212.0
경기도     수원시 권선구      61  23939.0      2.0   93291.0   4725.0   1944.0
경상남도    거제시          27  26151.0     17.0   75012.0   5921.0   1563.0
인천광역시   연수구          30  16985.0      4.0   71265.0   3459.0   1485.0
서울특별시   구로구          54   9119.0      3.0   67649.0   2570.0   1409.0
경기도     과천시          31  12504.0      5.0   64617.0   4207.0   1346.0
        용인시 기흥구      41  36024.0      2.0   64196.0   5959.0   1337.0
강원도     홍천군           8  30159.0     11.0   58839.0  11150.0   1226.0
경기도     고양시 일산동구    113  15101.0      3.0   57047.0   1695.0   1188.0
        연천군           5  26017.0    336.0   54695.0  12689.0   1139.0
서울특별시   중구           68  14789.0     12.0   54484.0   2021.0   1135.0
        강남구          72   5734.0      2.0   52179.0   1396.0   1087.0
        성북구          83   8131.0      3.0   44227.0   1321.0    921.0
전라북도    전주시 덕진구      97  10782.0      7.0   40556.0   1197.0    845.0
서울특별시   강서구          39   46

In [64]:
g_e2=round(df_e.groupby('시도')['노출'].agg(['sum']),0)
g_e2.sort_values('sum', ascending=False)

,sum
시도,
경기도,7426624.0
서울특별시,3118389.0
인천광역시,1598427.0
부산광역시,673229.0
충청남도,587066.0
강원도,503767.0
전라북도,382528.0
충청북도,257787.0
광주광역시,225314.0
